# <span style="color:Red">Modelos Factoriales</span>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm
from scipy.optimize import minimize
import function.F_Datos as FDatos
import function.F_FamaFactors as gff

In [4]:

retornos = pd.read_csv('data/returns_selected.csv',index_col=0)
retornos.index.names = ['Date']
retornos.index = pd.to_datetime(retornos.index)

In [ ]:
retornos.head()

,LU0272423913,LU0474536744,LU0147408487,LU1160359102,LU1160358633,IE0004866889,LU0966156712,LU1048657123,LU0966156399,IE00B03HCY54,...,IE00BLY1N519,IE00BLY27S52,LU0963986467,LU0858297186,IE00BLWF5D72,IE00BLWF5F96,IE00BHCR9339,IE00BCRY5R01,LU0858297772,LU0963987275
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,-0.014990,-0.002037,-0.015796,-0.024757,-0.024781,-0.004787,-0.012550,-0.012539,-0.012622,0.0,...,-0.001022,-0.000932,0.000582,0.000619,-0.001010,-0.001013,0.000707,0.000683,0.000507,0.000606
2016-01-07,-0.022914,-0.040227,-0.022918,-0.037041,-0.037004,-0.051189,-0.025975,-0.025978,-0.025975,0.0,...,-0.001023,-0.001222,0.001066,-0.001134,-0.002022,-0.001014,-0.001566,-0.001588,-0.001014,0.001110
2016-01-08,-0.011099,0.008457,-0.003766,-0.017866,-0.017866,0.006274,-0.014878,-0.014899,-0.014929,0.0,...,-0.001024,-0.000833,-0.001260,-0.001032,0.000000,0.000000,0.000868,0.000812,-0.001015,-0.001110
2016-01-11,-0.013483,-0.034265,-0.019048,0.000870,0.000879,-0.038846,-0.012839,-0.012888,-0.012988,0.0,...,0.001024,0.000118,0.000679,-0.000930,0.000000,0.000000,-0.001010,-0.001026,-0.001016,0.000606
2016-01-12,-0.001132,0.004348,0.005753,0.005698,0.005757,0.005135,0.003064,0.003002,0.002993,0.0,...,0.000000,-0.000168,-0.001261,-0.001241,0.000000,0.000000,-0.000453,-0.000485,-0.001221,-0.001111


In [6]:
retornos_ff = retornos.copy()

## <span style="color:white">French and Fama</span>



In [7]:
#Obtencion de datos de Fama-French con frecuencia mensual
ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)

In [8]:
ff3_monthly.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-31,0.0289,-0.0255,-0.0239,0.0022
1926-08-31,0.0264,-0.0114,0.0381,0.0025
1926-09-30,0.0038,-0.0136,0.0005,0.0023
1926-10-31,-0.0327,-0.0014,0.0082,0.0032
1926-11-30,0.0254,-0.0011,-0.0061,0.0031


In [9]:
#Resampleo
retornos_ff = retornos_ff.resample('ME').sum()
ff_data     = ff3_monthly.merge(retornos_ff,on='Date')

In [10]:
ff_data.head()

,Mkt-RF,SMB,HML,RF,LU0272423913,LU0474536744,LU0147408487,LU1160359102,LU1160358633,IE0004866889,...,IE00BLY1N519,IE00BLY27S52,LU0963986467,LU0858297186,IE00BLWF5D72,IE00BLWF5F96,IE00BHCR9339,IE00BCRY5R01,LU0858297772,LU0963987275
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-31,-0.0574,-0.0340,0.0208,0.0001,-0.034598,-0.093074,-0.073783,-0.098300,-0.098034,-0.115114,...,-2.044990e-03,-0.002662,-0.010042,-0.006104,-0.001010,-1.001803e-16,-0.008401,-0.008935,-0.005799,-0.008933
2016-02-29,-0.0006,0.0073,-0.0061,0.0002,0.037781,-0.023353,-0.010437,-0.012362,-0.011971,-0.017168,...,-4.000706e-16,0.000158,-0.004321,-0.008755,0.002018,2.022245e-03,-0.006546,-0.007125,-0.008299,-0.003473
2016-03-31,0.0695,0.0075,0.0122,0.0002,0.095888,0.117783,0.069037,0.039416,0.039669,0.051974,...,1.523646e-02,0.014879,0.026417,0.006157,0.016992,1.702595e-02,0.024755,0.024125,0.006972,0.028049
2016-04-30,0.0092,0.0077,0.0322,0.0001,0.063395,0.004731,-0.034873,0.042598,0.042936,0.000567,...,2.014099e-03,0.001202,0.013708,0.002494,0.003956,3.964326e-03,0.012797,0.012176,0.003264,0.015108
2016-05-31,0.0178,-0.0018,-0.0162,0.0001,-0.003620,-0.017001,0.042190,0.012212,0.012618,0.005159,...,1.299369e-02,0.012678,0.000473,0.003108,0.014699,1.472781e-02,0.003562,0.002927,0.003761,0.001860


### <span style="color:white">Obtencion de los Facotores de Fama-French</span>

In [11]:
activo = 'LU0272423913'

In [12]:
# Factores con un modelo de regresion lineal
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data[activo] - ff_data['RF']
X = sm.add_constant(X)
ff_model = sm.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     122.8
Date:                Sat, 05 Jul 2025   Prob (F-statistic):           2.84e-26
Time:                        11:22:41   Log-Likelihood:                 159.04
No. Observations:                  67   AIC:                            -310.1
Df Residuals:                      63   BIC:                            -301.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0080      0.003     -2.630      0.0

In [13]:
ff_model.params

const    -0.007963
Mkt-RF    1.066390
SMB       0.155483
HML       0.427546
dtype: float64

In [14]:
rf             = ff_data['RF'].mean()
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium   = ff3_monthly['SMB'].mean()
value_premium  = ff3_monthly['HML'].mean()

expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 
expected_yearly_return = expected_monthly_return * 12
print("Expected monthly return:", expected_monthly_return)
print("Expected yearly return: " + str(expected_yearly_return))

Expected monthly return: 0.009794835203876162
Expected yearly return: 0.11753802244651396


### <span style="color:white">Evaluacion de las estrategias</span>

In [15]:
params = pd.DataFrame(columns=['intercept', 'b1', 'b2', 'b3'])

for col in ff_data.columns[4:]:
    X = ff_data[['Mkt-RF', 'SMB', 'HML']]
    y = ff_data[col] - ff_data['RF']
    X = sm.add_constant(X)
    ff_model = sm.OLS(y, X).fit()
    intercept, b1, b2, b3 = ff_model.params
    params.loc[col] =  intercept, b1, b2, b3

params.head()

,intercept,b1,b2,b3
LU0272423913,-0.007963,1.066390,0.155483,0.427546
LU0474536744,-0.001023,0.783100,0.240714,-0.167153
LU0147408487,0.003651,1.072965,0.214652,-0.629265
LU1160359102,-0.009553,0.975150,0.279212,0.545974
LU1160358633,-0.009244,0.974934,0.279762,0.545412


In [16]:
fig = go.Figure(data=[go.Scatter3d(
    x=params['b1'],
    y=params['b2'],
    z=params['b3'],
    mode='markers',
    marker=dict(
        size=5,
        color=params['b1'],  # Color para el mercado
        colorscale='Viridis',
        opacity=0.8
    ),
    text=params.index,  #Nombre de los fondos
    hoverinfo='text'  
)])

fig.update_layout(
    scene=dict(
        xaxis_title='Market Premium',
        yaxis_title='Size Premium',
        zaxis_title='Value Premium',
        bgcolor='white'  
    ),
    margin=dict(l=0, r=0, b=0, t=0),
    height=700  
)

fig.show()



In [17]:

retornos_estrg = pd.read_csv('data/returns_strategies.csv',index_col=0)
retornos_estrg.index.names = ['Date']
retornos_estrg.index = pd.to_datetime(retornos_estrg.index)

In [18]:
retornos_estrg = retornos_estrg.resample('ME').sum()
ff_data_estrg     = ff3_monthly.merge(retornos_estrg,on='Date')

In [19]:
ff_data_estrg.head()

,Mkt-RF,SMB,HML,RF,Equally Weighted,Momentum 11M,Momentum 1M
Date,,,,,,,
2017-01-31,0.0194,-0.0116,-0.0276,0.0004,0.009885,0.000043,0.016697
2017-02-28,0.0355,-0.0215,-0.0150,0.0004,0.019965,0.037953,0.038971
2017-03-31,0.0017,0.0113,-0.0333,0.0003,0.008535,0.000685,0.026026
2017-04-30,0.0108,0.0066,-0.0205,0.0005,0.005362,0.008857,0.029557
2017-05-31,0.0107,-0.0252,-0.0382,0.0006,0.005023,0.038856,0.036274


In [20]:
params_estrg = pd.DataFrame(columns=['intercept', 'b1', 'b2', 'b3'])
for col in ff_data_estrg.columns[4:]:
    X = ff_data_estrg[['Mkt-RF', 'SMB', 'HML']]
    y = ff_data_estrg[col] - ff_data_estrg['RF']
    X = sm.add_constant(X)
    ff_model = sm.OLS(y, X).fit()
    intercept, b1, b2, b3 = ff_model.params
    params_estrg.loc[col] =  intercept, b1, b2, b3

params_estrg.head()

,intercept,b1,b2,b3
Equally Weighted,-0.002206,0.489150,0.122091,0.141262
Momentum 11M,0.001363,0.817258,0.176371,-0.109547
Momentum 1M,0.000224,0.549389,-0.069151,-0.014608


In [21]:
fig = go.Figure(data=[go.Scatter3d(
    x=params_estrg['b1'],
    y=params_estrg['b2'],
    z=params_estrg['b3'],
    mode='markers',
    marker=dict(
        size=5,
        color=params_estrg['b1'],  
        colorscale='Viridis',
        opacity=0.8
    ),
    text=params_estrg.index,  
    hoverinfo='text'  
)])

fig.update_layout(
    scene=dict(
        xaxis_title='Market Premium',
        yaxis_title='Size Premium',
        zaxis_title='Value Premium',
        bgcolor='white'  
    ),
    margin=dict(l=0, r=0, b=0, t=0),
    height=700  
)

fig.show()

### <span style="color:white">Retornos y Factores</span>

In [22]:
retornos_carhart = retornos.copy()

In [23]:
ff4_monthly = gff.carhart4Factor(frequency='m')
ff4_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff4_monthly.set_index('Date', inplace=True)

In [24]:
retornos_carhart = retornos_carhart.resample('ME').sum()
ff_carhart     = ff4_monthly.merge(retornos_carhart,on='Date')

In [25]:
ff_carhart.head()

,Mkt-RF,SMB,HML,RF,MOM,LU0272423913,LU0474536744,LU0147408487,LU1160359102,LU1160358633,...,IE00BLY1N519,IE00BLY27S52,LU0963986467,LU0858297186,IE00BLWF5D72,IE00BLWF5F96,IE00BHCR9339,IE00BCRY5R01,LU0858297772,LU0963987275
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-31,-0.0574,-0.0340,0.0208,0.0001,0.0160,-0.034598,-0.093074,-0.073783,-0.098300,-0.098034,...,-2.044990e-03,-0.002662,-0.010042,-0.006104,-0.001010,-1.001803e-16,-0.008401,-0.008935,-0.005799,-0.008933
2016-02-29,-0.0006,0.0073,-0.0061,0.0002,-0.0435,0.037781,-0.023353,-0.010437,-0.012362,-0.011971,...,-4.000706e-16,0.000158,-0.004321,-0.008755,0.002018,2.022245e-03,-0.006546,-0.007125,-0.008299,-0.003473
2016-03-31,0.0695,0.0075,0.0122,0.0002,-0.0504,0.095888,0.117783,0.069037,0.039416,0.039669,...,1.523646e-02,0.014879,0.026417,0.006157,0.016992,1.702595e-02,0.024755,0.024125,0.006972,0.028049
2016-04-30,0.0092,0.0077,0.0322,0.0001,-0.0598,0.063395,0.004731,-0.034873,0.042598,0.042936,...,2.014099e-03,0.001202,0.013708,0.002494,0.003956,3.964326e-03,0.012797,0.012176,0.003264,0.015108
2016-05-31,0.0178,-0.0018,-0.0162,0.0001,0.0147,-0.003620,-0.017001,0.042190,0.012212,0.012618,...,1.299369e-02,0.012678,0.000473,0.003108,0.014699,1.472781e-02,0.003562,0.002927,0.003761,0.001860


In [26]:
params = pd.DataFrame(columns=['intercept', 'b1', 'b2', 'b3','b4'])
for col in ff_carhart.columns[5:]:
    X = ff_carhart[['Mkt-RF', 'SMB', 'HML', 'MOM']]
    y = ff_carhart[col] - ff_carhart['RF']
    X = sm.add_constant(X)
    ff_model = sm.OLS(y, X).fit()
    intercept, b1, b2, b3, b4 = ff_model.params
    params.loc[col] =  intercept, b1, b2, b3, b4

params.head()

,intercept,b1,b2,b3,b4
LU0272423913,-0.008023,1.024096,0.121975,0.353952,-0.147123
LU0474536744,-0.001022,0.784394,0.241739,-0.164902,0.004499
LU0147408487,0.003751,1.144738,0.271514,-0.504375,0.249668
LU1160359102,-0.009638,0.914379,0.231066,0.440228,-0.211396
LU1160358633,-0.009329,0.914286,0.231713,0.439879,-0.210971


In [27]:
params['market_premium'] = params['b1'] > 0.8
params['size_premium'] = params['b2'] > 0.8
params['value_premium'] = params['b3'] > 0.8
params['momentum'] = params['b4'] > 0.8

params

,intercept,b1,b2,b3,b4,market_premium,size_premium,value_premium,momentum
LU0272423913,-0.008023,1.024096,0.121975,0.353952,-0.147123,True,False,False,False
LU0474536744,-0.001022,0.784394,0.241739,-0.164902,0.004499,False,False,False,False
LU0147408487,0.003751,1.144738,0.271514,-0.504375,0.249668,True,False,False,False
LU1160359102,-0.009638,0.914379,0.231066,0.440228,-0.211396,True,False,False,False
LU1160358633,-0.009329,0.914286,0.231713,0.439879,-0.210971,True,False,False,False
IE0004866889,0.000069,0.639776,0.504455,-0.158717,0.053159,False,False,False,False
LU0966156712,-0.001864,0.798717,0.243270,0.441828,-0.342465,False,False,False,False
LU1048657123,-0.002664,0.802240,0.233181,0.441647,-0.342136,True,False,False,False
LU0966156399,-0.003259,0.798818,0.237220,0.440362,-0.344118,False,False,False,False
IE00B03HCY54,-0.001633,1.024484,0.501022,0.082816,0.115962,True,False,False,False


In [28]:
ff_data_estrg_carhart     = ff4_monthly.merge(retornos_estrg,on='Date')

In [29]:
ff_data_estrg_carhart.head()

,Mkt-RF,SMB,HML,RF,MOM,Equally Weighted,Momentum 11M,Momentum 1M
Date,,,,,,,,
2017-01-31,0.0194,-0.0116,-0.0276,0.0004,-0.0093,0.009885,0.000043,0.016697
2017-02-28,0.0355,-0.0215,-0.0150,0.0004,-0.0151,0.019965,0.037953,0.038971
2017-03-31,0.0017,0.0113,-0.0333,0.0003,-0.0097,0.008535,0.000685,0.026026
2017-04-30,0.0108,0.0066,-0.0205,0.0005,0.0048,0.005362,0.008857,0.029557
2017-05-31,0.0107,-0.0252,-0.0382,0.0006,0.0150,0.005023,0.038856,0.036274


In [30]:
params_estrg_carhart = pd.DataFrame(columns=['intercept', 'b1', 'b2', 'b3','b4'])
for col in ff_data_estrg_carhart.columns[5:]:
    X = ff_data_estrg_carhart[['Mkt-RF', 'SMB', 'HML', 'MOM']]
    y = ff_data_estrg_carhart[col] - ff_data_estrg_carhart['RF']
    X = sm.add_constant(X)
    ff_model = sm.OLS(y, X).fit()
    intercept, b1, b2, b3, b4 = ff_model.params
    params_estrg_carhart.loc[col] =  intercept, b1, b2, b3, b4

In [31]:
params_estrg_carhart['market_premium'] = params_estrg_carhart['b1'] > 0.8
params_estrg_carhart['size_premium']   = params_estrg_carhart['b2'] > 0.8
params_estrg_carhart['value_premium']  = params_estrg_carhart['b3'] > 0.8
params_estrg_carhart['momentum']       = params_estrg_carhart['b4'] > 0.8

params_estrg_carhart.head()


,intercept,b1,b2,b3,b4,market_premium,size_premium,value_premium,momentum
Equally Weighted,-0.002205,0.482212,0.116730,0.128580,-0.024852,False,False,False,False
Momentum 11M,0.001349,0.906820,0.245571,0.054153,0.320792,True,False,False,False
Momentum 1M,0.000219,0.578509,-0.046651,0.038617,0.104302,False,False,False,False
